In [ ]:
import os
os.system('pip install /kaggle/input/kerasapplications -q')
os.system('pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps')

from efficientnet.tfkeras import EfficientNetB4

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, cv2, math, re
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import random

#model imports (keras/tensorflow)
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import layers, models
# from tensorflow.keras.applications import EfficientNetB0
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow.keras.backend as K
#from tensorflow.keras.applications import ResNet50

from kaggle_datasets import KaggleDatasets
from functools import partial

print("Tensorflow version " + tf.__version__)

# Prediction

In [ ]:
WORK_DIR = '../input/cassava-leaf-disease-classification'
TARGET_SIZE = 512

We have a few options for model submits below. There are multiple variations due to various model blending techniques. 

In [ ]:
%%time
# single model submit. NOTE: compile = false for models with custom loss functions
# model = keras.models.load_model('../input/cldmanualweights/EffNet_untrained_TPU_model (1).h5', compile=False)
# model.load_weights('../input/cldmanualweights/Effnet_TPU_Model_best_weights.h5')

#two_fold
# model_0 = keras.models.load_model('../input/cldbestsinglemodels/Archive/0.888 effnet b3 submit 54/EffNetB3_untrained_TPU_model (7).h5', compile = False)
# model_1 = keras.models.load_model('../input/cldbestsinglemodels/Archive/0.888 v58/EffNetB3_untrained_TPU_model (7).h5', compile = False)

# model_0.load_weights('../input/cldbestsinglemodels/Archive/0.888 effnet b3 submit 54/EffNetB3_TPU_Model_best_weights (7).h5')
# model_1.load_weights('../input/cldbestsinglemodels/Archive/0.888 v58/EffNetB3_TPU_Model_best_weights (7).h5')

#five_fold
model_0 = keras.models.load_model('../input/cldmanualweights/EffNet_untrained_TPU_model.h5', compile = False)
model_1 = keras.models.load_model('../input/cldmanualweights/EffNet_untrained_TPU_model.h5', compile = False)
model_2 = keras.models.load_model('../input/cldmanualweights/EffNet_untrained_TPU_model.h5', compile = False)
model_3 = keras.models.load_model('../input/cldmanualweights/EffNet_untrained_TPU_model.h5', compile = False)
model_4 = keras.models.load_model('../input/cldmanualweights/EffNet_untrained_TPU_model.h5', compile = False)

model_0.load_weights('../input/cldmanualweights/Model_0_best_weights.h5')
model_1.load_weights('../input/cldmanualweights/Model_1_best_weights.h5')
model_2.load_weights('../input/cldmanualweights/Model_2_best_weights.h5')
model_3.load_weights('../input/cldmanualweights/Model_3_best_weights.h5')
model_4.load_weights('../input/cldmanualweights/Model_4_best_weights.h5')

In [ ]:
sample_submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
sample_submission

Variables/Notes

- Need to make sure how to center crop the test_images on submission or resizing based on how the model was trained
- Also need to make sure I am normalizing the pixels the same way in Test and Training

In [ ]:
NEW_WIDTH = 512
NEW_HEIGHT = 512
width = 800
height = 600
left = (width - NEW_WIDTH)/2
top = (height - NEW_HEIGHT)/2
right = (width + NEW_WIDTH)/2
bottom = (height + NEW_HEIGHT)/2

### Making Prediction w/ Single Model

- LB: 0.887 effnetB3

In [ ]:
# %%time
# #making preds on all the test_images
# preds = []

# for image_id in sample_submission.image_id:
#     image = Image.open(os.path.join(WORK_DIR,  "test_images", image_id))
#     image = image.crop((left, top, right, bottom))  
#     image = np.expand_dims(image, axis = 0)
#     image = image / 255 #normalizing pixel vals before predicting
#     image[0,:,:,0] = (image[0,:,:,0] - 0.4303)/ 0.2142 #TEST
#     image[0,:,:,1] = (image[0,:,:,1] - 0.4967)/ 0.2191 #TEST
#     image[0,:,:,2] = (image[0,:,:,2] - 0.3134)/ 0.1954 #TEST
#     preds.append(np.argmax(model.predict(image)))

# sample_submission['label'] = preds

# sample_submission.to_csv('submission.csv', index = False)

# sample_submission

### Making Predictions w/ Highest Confidence Sum

- LB: 0.881 w/ cldfiveweights v1

In [ ]:
%%time
preds = []

for image_id in sample_submission.image_id:
    image = Image.open(os.path.join(WORK_DIR,  "test_images", image_id))
    image = image.resize((512, 512), Image.ANTIALIAS)
    image = np.expand_dims(image, axis = 0)
    image = image / 255 #normalizing pixel vals before predicting
    image[0,:,:,0] = (image[0,:,:,0] - 0.4303)/ 0.2142 #TEST
    image[0,:,:,1] = (image[0,:,:,1] - 0.4967)/ 0.2191 #TEST
    image[0,:,:,2] = (image[0,:,:,2] - 0.3134)/ 0.1954 #TEST
    
    model_0_pred = model_0.predict(image)
    model_1_pred = model_1.predict(image)
    model_2_pred = model_2.predict(image)
    model_3_pred = model_3.predict(image)
    model_4_pred = model_4.predict(image)
    
    all_preds = (model_0_pred + model_1_pred + model_2_pred + model_3_pred + model_4_pred)
    preds.append(np.argmax(all_preds))
    
sample_submission['label'] = preds

sample_submission.to_csv('submission.csv', index = False)

sample_submission

### Making Predictions w/ Highest Pred Confidence

- LB: 0.888 w/ (CLD Best Models (0.883 + 0.881)

I think this submission type will be handy if I use more than model type (ie. resnet + effnet)

In [ ]:
# %%time
# preds = []

# for image_id in sample_submission.image_id:
#     image = Image.open(os.path.join(WORK_DIR,  "test_images", image_id))
#     image = image.crop((left, top, right, bottom))  
#     image = np.expand_dims(image, axis = 0)
#     image = image / 255 #normalizing pixel vals before predicting
#     image[0,:,:,0] = (image[0,:,:,0] - 0.4303)/ 0.2142 #TEST
#     image[0,:,:,1] = (image[0,:,:,1] - 0.4967)/ 0.2191 #TEST
#     image[0,:,:,2] = (image[0,:,:,2] - 0.3134)/ 0.1954 #TEST
    
#     model_0_pred = model_0.predict(image)
#     model_1_pred = model_1.predict(image)
#     model_2_pred = model_2.predict(image)
#     model_3_pred = model_3.predict(image)
#     model_4_pred = model_4.predict(image)
    
#     highest_confidence = 0
#     preds_list = [model_0_pred, model_1_pred, model_2_pred, model_3_pred, model_4_pred]

#     for pred_group in preds_list:
#         if highest_confidence == 0:
#             highest_confidence = float(np.min(pred_group))

#         if float(np.max(pred_group)) > highest_confidence:
#             highest_confidence = float(np.max(pred_group))
#             pred = np.argmax(pred_group)
    
#     preds.append(pred)
    
# sample_submission['label'] = preds

# sample_submission.to_csv('submission.csv', index = False)

# sample_submission